In [43]:
import math
import os

import pandas as pd

import rvms

csv_dir = '../../../csvFiles/'
if csv_dir == '':
    raise ValueError('Specificare un percorso valido!')

def load_csv(path: str) -> pd.DataFrame:
    return pd.read_csv(path, sep='\\s*,\\s*', engine='python')

def confidence_interval(values: list[float], loc: float=0.95) -> float:
    n: int = len(values)
    if n <= 1:
        raise ValueError("list of 2+ elements required")
    mean = sum(values) / n
    std = sum(map(lambda x: (x - mean)**2, values)) / n
    u = 1.0 - 0.5 * (1.0 - loc)
    t = rvms.idfStudent(n - 1, u)
    return t * std / math.sqrt(n - 1)
    

In [44]:
from collections import defaultdict


def format_center_name(s: str) -> str:
    return ''.join(w[0] for w in s.split('_'))

def sci(x: float, nf: int=4) -> str:
    if x == 0:
        return str(x)
    if x < 1:
        exp = 0
        while x < 1:
            x *= 10
            exp += 1
        return f'{x:.4f} \\cdot 10^{{-{exp}}}'
    else:
        return f'{x:.4f}'

def create_table(
        parent_dir: str,
        simulation_type = ('FINITE_SIMULATION', 'IMPROVED_FINITE_SIMULATION'),
        indexes = ('E[Ns]', 'E[Ts]', 'E[Nq]', 'E[Tq]', 'E[s]', 'ρ', 'λ')):
    data: list[list[str]] = []
    # ['Reparto Istruttorie', 'Scoring Automatico', 'Comitato Credito', 'Reparto Liquidazioni']
    for sim in simulation_type:
        parent = os.path.join(csv_dir, sim, 'results')
        for file_name in os.listdir(parent):
            center_name: str = file_name.replace('.csv', '')
            path = os.path.join(parent, file_name)
            df: pd.DataFrame = load_csv(path)
            row: list[str] = [format_center_name(center_name)]
            for i in indexes:
                x = df[i].mean()
                ci = confidence_interval(df[i])
                if ci == 0:
                    cell = '-'
                else:
                    cell = f'${x:.4f}\\pm{sci(ci)}$'
                row.append(cell)
            data.append(row)
        columns = ['Centro']
        for i in indexes:
            if i == 'ρ':
                s = '$\\rho$'
            elif i == 'λ':
                s = '$\\lambda$'
            else:
                s = f'${i}$'
            columns.append(s)
        table = pd.DataFrame(data, columns=columns)
        path: str = os.path.join(parent_dir, f'{sim}_output.tex')        
        table.style.hide(axis="index").to_latex(path, hrules=True)
        data.clear()

def create_batch_table(
        parent_dir: str,
        simulation_type =('BATCH_SIMULATION_EXPONENTIAL', 'IMPROVED_BATCH_SIMULATION_EXPONENTIAL'),
        indexes = ('E[Ns]', 'E[Ts]', 'E[Nq]', 'E[Tq]', 'E[s]', 'ρ', 'λ')):
    data: dict[str, list[str]] = defaultdict(list)
    
    for i in indexes:
            if i == 'ρ':
                s = '$\\rho$'
            elif i == 'λ':
                s = '$\\lambda$'
            else:
                s = f'${i}$'
            data['Indice'].append(f'{s} analitico')
            data['Indice'].append(f'{s} misurato')
            data['Indice'].append('CI')
    
    # ['Reparto Istruttorie', 'Scoring Automatico', 'Comitato Credito', 'Reparto Liquidazioni']
    for sim in simulation_type:
        a_path = os.path.join(csv_dir, sim, 'analyticalResults', 'analyticalResults.csv')
        a_df: pd.DataFrame = load_csv(a_path)
        parent = os.path.join(csv_dir, sim, 'results')
        for file_name in os.listdir(parent):
            center_name: str = file_name.replace('.csv', '')
            path = os.path.join(parent, file_name)
            a_row = a_df[a_df['Center'] == center_name].iloc[0]
            df: pd.DataFrame = load_csv(path)
            center_name = center_name.replace('_', '\\_')
            for i in indexes:
                x = df[i].mean()
                ci = confidence_interval(df[i])
                if ci == 0:
                    cell = '-'
                else:
                    cell = f'$\\pm {sci(ci)}$'
                data[center_name].append(f'${a_row[i]}$')
                data[center_name].append(f'${x}$')
                data[center_name].append(cell)
        table = pd.DataFrame(data)
        path: str = os.path.join(parent_dir, f'{sim}_output.tex')        
        table.style.hide(axis="index").to_latex(path, hrules=True)
        data.clear()

In [45]:
create_table('tex',
             indexes=['E[Ns]', 'E[Ts]', 'E[Nq]', 'E[Tq]', 'ρ', 'λ'])